In [118]:
import numpy as np
import pandas as pd

# Load data

Get the text from the txt file

In [119]:
data = '../data/historia.txt'

# Open the file in read mode
with open(file_path, "r") as file:
    # Read the contents of the file
    data = file.read()

1. Load data for lang chain

In [182]:
from langchain.document_loaders import TextLoader

In [183]:
loader = TextLoader("../data/historia.txt")
documents = loader.load()

2. Split the text

In [184]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [185]:
splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=200, length_function = len)
splits = splitter.split_documents(documents)

3. Generate the embeddings

In [186]:
from langchain.embeddings import SentenceTransformerEmbeddings

In [187]:
embedding = SentenceTransformerEmbeddings(model_name="allenai/longformer-base-4096")

No sentence-transformers model found with name C:\Users\LP00035/.cache\torch\sentence_transformers\allenai_longformer-base-4096. Creating a new one with MEAN pooling.


Verify the lenght of the embedding

In [188]:
embedding.client

SentenceTransformer(
  (0): Transformer({'max_seq_length': 4098, 'do_lower_case': False}) with Transformer model: LongformerModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

4. Generate the dense database

In [189]:
from langchain.vectorstores import Chroma

In [191]:
NAME_VECTOR_STORE = 'embeddings-longformer'
vectorstore = Chroma.from_documents(documents=splits, embedding=embedding, persist_directory=NAME_VECTOR_STORE)

Make that vector store will be saved in the disk

In [192]:
vectorstore.persist()

Load the vector store from the disk

In [193]:
vectorstore = Chroma(persist_directory=NAME_VECTOR_STORE, embedding_function=embedding)

Make a question

In [194]:
query = 'Quien es Sebastian Sarasti'
docs = vectorstore.similarity_search_with_score(query, topn=5)

5. Make a retriever

In [195]:
from langchain.chains import RetrievalQAWithSourcesChain

Create a retriever from the chroma db

In [196]:
retriever_chroma = vectorstore.as_retriever(search_kwargs={'k': 10})

In [197]:
retriever_chroma.get_relevant_documents(query='Que es E-SAS', topn=10)

[Document(page_content='Nikola se convirtió en el "probador oficial" de productos de E-SAS. Cada nuevo producto que la empresa consideraba agregar a su catálogo pasaba por un riguroso proceso de prueba, con Nikola como el juez principal. Sebastián estaba decidido a garantizar que cada producto que ofreciera en su tienda en línea cumpliera con los más altos estándares de calidad y comodidad, y nadie mejor que Nikola para ayudar en esa tarea.', metadata={'source': '../data/historia.txt'}),
 Document(page_content='Sebastián Alejandro Sarasti, el fundador de E-SAS, siempre había sido un apasionado por la tecnología. Desde joven, tenía una afinidad especial por las computadoras y el mundo digital. Su amor por la programación y la inteligencia artificial lo llevaron a un viaje de autodescubrimiento y aprendizaje que finalmente se fusionaría con su amor inquebrantable por los animales.', metadata={'source': '../data/historia.txt'}),
 Document(page_content='Pero Sebastián también se dio cuenta

Get a LLM

In [198]:
from langchain.chat_models import ChatOpenAI

In [199]:
# load openai key
with open('../openaikey.txt', "r") as file:
    # Read the contents of the file
    OPENAI_KEY = file.read()

In [206]:
llm = ChatOpenAI(openai_api_key=OPENAI_KEY, model_name='gpt-3.5-turbo', temperature=0, max_tokens=250)

In [207]:
qa_chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever=retriever_chroma
)

In [208]:
respuesta = qa_chain('Quien es Nikola')

In [209]:
respuesta

{'question': 'Quien es Nikola',
 'answer': 'Nikola es el "probador oficial" de productos de E-SAS. Nikola ayuda a garantizar que cada producto cumpla con los estándares de calidad y comodidad de la empresa. Nikola es el juez principal en el proceso de prueba de nuevos productos. \n',
 'sources': '../data/historia.txt'}